In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import d2l
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#下面进入backbone网络模块实现
#基础模块-Conv模块
class Conv(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=None,groups=1,activation=None):
        super().__init__()
        if padding is None:
            padding=kernel_size//2
        
        self.conv=nn.Conv2d(in_channels,out_channels,
                            kernel_size,stride,padding,
                            groups=groups,bias=False)
        self.bn=nn.BatchNorm2d(out_channels)
        self.act=nn.SiLU() if activation else nn.Identity()
    
    def foreard(self,x):
        return self.act(self.bn(self.conv(x)))

#瓶颈模块
class BottleNeck(nn.Module):
    def __init__(self, in_channels,out_channels,shortcut=True):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,1,1)
        self.conv2=nn.Conv2d(out_channels,out_channels,3,1)
        self.shortcut=shortcut and in_channels==out_channels
    
    def forward(self,x):
        if self.shortcut:
            return x+self.conv2(self.conv1(x))
        else:
            return self.conv2(self.conv1(x))

#C3模块
class C3(nn.Module):
    def __init__(self, in_channels,out_channels,n=1,shortcut=True):
        super().__init__()
        hidden_channels=out_channels//2
        #两路卷积，一路只通过卷积层，一路通过n个瓶颈层
        self.conv1=nn.Conv2d(in_channels,hidden_channels,1,1)
        self.conv2=nn.Conv2d(in_channels,hidden_channels,1,1)
        self.conv3=nn.Conv2d(2*hidden_channels,out_channels,1,1)
        self.m=nn.Sequential(*[BottleNeck(hidden_channels,hidden_channels,shortcut) for _ in range(n)])
    
    def forward(self,x):
        x1=self.conv1(x)
        x2=self.conv2(x)
        x1=self.m(x1)
        x=torch.cat([x1,x2],dim=1)
        return self.conv3(x)
    
#SPPF，空间金字塔池化模块
class SPPF(nn.Module):
    def __init__(self,in_channels,out_channels,k=5):
        super().__init__()
        hidden_channels=out_channels//2
        self.conv1=nn.Conv2d(in_channels,hidden_channels,1,1)
        self.conv2=nn.Conv2d(hidden_channels*4,out_channels,1,1)
        self.pool=nn.MaxPool2d(kernel_size=k,stride=1,padding=k//2)
    
    def forward(self,x):
        x=self.conv1(x)
        #串联三次池化层
        y1=x
        y2=self.pool(x)
        y3=self.pool(y2)#等同于9X9池化
        y4=self.pool(y3)#等同于13X13池化
        out=torch.cat([y1,y2,y3,y4],dim=1)
        return self.conv2(out)